# Imports

In [4]:
!pip install -U torchvision

Defaulting to user installation because normal site-packages is not writeable


In [52]:
from torchvision import datasets
from IPython.display import display
import torchvision.transforms.functional as F
import os
import numpy as np
from PIL import Image
import cv2
import Augmentor

## Gaussian Pyramid Levels

In [27]:
def get_level(img, times):
    out = img.copy()
    while times > 0:
        out = cv2.pyrDown(out)
        times -= 1
    return out

In [28]:
train_dataset = datasets.OxfordIIITPet(root="pet", download=True)

In [29]:
classes = set()
for sample, label in train_dataset:
    classes.add(label)

print(classes)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36}


In [64]:
def get_count(parent):
    count = 0
    for f in os.listdir(parent):
        if os.path.isfile(os.path.join(parent, f)):
            count += 1
    print(f"Number of files: {count}")
    return

In [65]:
# collect image paths
src = "pet/oxford-iiit-pet/images"

paths = []
for f in sorted(os.listdir(src)):
    img = os.path.join(src, f)
    if img.endswith('.jpg'):
        paths.append(img)

In [66]:
print(f"Number of images: {len(paths)}")

Number of images: 29560


In [67]:
# create gaussian pyramids of the images
parent = src
dst = "pet/train"

# iterate through the directory
count = 0
for f in sorted(os.listdir(parent)):
    fpath = os.path.join(parent, f)
    if not fpath.endswith("_d.jpg") and fpath.endswith(".jpg"):
        
        # read in the image
        img = np.array(Image.open(fpath))
        
        fpathnew = os.path.join(dst, f"{count}.jpg")
        count += 1

        # convert to BGR
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        if type(img) == type(None):
            print(fpath)

        # downsample once, twice, thrice
        for i in range(1, 4):
            dimg = get_level(img, i)
            dpath = fpathnew.replace(".jpg", f"_{i}_d.jpg")
            cv2.imwrite(dpath, dimg)
        cv2.imwrite(fpathnew, img)

In [68]:
# images after the gaussian images have been added
get_count(dst)

Number of files: 29560


In [69]:
# augment the images

p = Augmentor.Pipeline(dst)
p.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
p.flip_left_right(probability=0.75)
p.crop_random(probability=0.5, percentage_area=0.85)
p.shear(probability=0.5, max_shear_left=10, max_shear_right=15)
p.random_distortion(probability=0.5, grid_width=8, grid_height=8, magnitude=8)

Initialised with 29560 image(s) found.
Output directory set to pet/train/output.

In [70]:
p.sample(30000)

Processing <PIL.Image.Image image mode=RGB size=425x319 at 0x7FA600C3A280>:  55%|█████▌    | 16517/30000 [03:26<00:16, 825.67 Samples/s]                   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Processing <PIL.Image.Image image mode=RGB size=125x94 at 0x7FA58D165550>:  70%|██████▉   | 20927/30000 [03:30<00:08, 1025.17 Samples/s]                   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Processing <PIL.Image.Image image mode=R

In [20]:
def get_triplets(parent):
    """ Given the parent directory form triplets using paths """
    
    data = {}
    
    flist = set()
    for f in os.listdir(parent):
        fpath = os.path.join(parent, f)
        flist.add(fpath)
    
    for f in os.listdir(parent):
        fpath = os.path.join(parent, f)
        if not fpath.endswith("_d.jpg"):
            
            data[fpath] = {"n": [], "p": []}
            
            # get the positives
            for i in range(1, 4):
                ppath = fpath.replace(".png", f"_{i}_d.png")
                data[fpath]["p"].append(ppath)
            
            # get the negatives
            # copy the entire file list
            data[fpath]["n"] = flist.copy()
            
            # get rid of the positive ones
            for i in data[fpath]["p"]:
                data[fpath]["n"].remove(i)
    return data

In [22]:
data = get_triplets("mnist/utrain")